In [2]:
%pip install -U transformers \
             datasets==2.14.4 \
             diffusers==0.20.0 \
             accelerate==0.21.0 \
             torch==2.0.1 \
             torchvision==0.15.2 \
             sentencepiece==0.1.99

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for datasets==2.14.4 from https://files.pythonhosted.org/packages/66/f8/38298237d18d4b6a8ee5dfe390e97bed5adb8e01ec6f9680c0ddf3066728/datasets-2.14.4-py3-none-any.whl.metadata
  Obtaining dependency information for diffusers==0.20.0 from https://files.pythonhosted.org/packages/24/4a/b45557513dfcfb6d69e5c138325a0d7431943792b662e482dcd886c11bb4/diffusers-0.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for accelerate==0.21.0 from https://files.pythonhosted.org/packages/70/f9/c381bcdd0c3829d723aa14eec8e75c6c377b4ca61ec68b8093d9f35fc7a7/accelerate-0.21.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 149.2 MB/s eta 0:00:00
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/13/c3/e942893f4864a424514c81640f114980cfd5aff7e7414d1e0255f4571111/xxhash-3.3.0-cp310-cp310-manylinux_2_17_x86_64.manyli

In [4]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "HuggingFaceM4/idefics-9b-instruct"
model = IdeficsForVisionText2Text.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)
processor = AutoProcessor.from_pretrained(model_name)

# Generation args
exit_condition = processor.tokenizer("<end_of_utterance>", add_special_tokens=False).input_ids
bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Zero-shot inference

![](https://hips.hearstapps.com/hmg-prod/images/dog-puns-1581708208.jpg)

In [10]:
url = "https://hips.hearstapps.com/hmg-prod/images/dog-puns-1581708208.jpg"
img = processor.image_processor.fetch_images([url])[0]

prompts = [
    "\nUser:",
    img,
    "Describe this image.\nAssistant: ",
]

inputs = processor(prompts, return_tensors="pt", debug=True).to(device)

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
generated_text

full_text='<s>\nUser:<fake_token_around_image><image><fake_token_around_image>Describe this image.\nAssistant:'


"<s> \nUser:<fake_token_around_image><image><fake_token_around_image> Describe this image.\nAssistant: The image features a small dog wearing a pair of black sunglasses, giving it a cool and stylish appearance. The dog is looking directly at the camera, capturing the viewer's attention. The sunglasses cover a significant portion of the dog's face, making it the central focus of the image. The dog appears to be a Boston Terrier, a breed known"

# One-shot Inference to guide the description using a complete example

![](https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg)
![](https://hips.hearstapps.com/hmg-prod/images/dog-puns-1581708208.jpg)

In [11]:
url = "https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg"
img = processor.image_processor.fetch_images([url])[0]

# Either use img or url
prompts = [
    "User:",
    img,
    "Describe this image."
    "Assistant: An image of two kittens in grass."
    "User:",
    "https://hips.hearstapps.com/hmg-prod/images/dog-puns-1581708208.jpg",
    "Describe this image.",
    "Assistant: "
]

inputs = processor(prompts, return_tensors="pt", debug=True).to(device)

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

full_text='<s>User:<fake_token_around_image><image><fake_token_around_image>Describe this image.Assistant: An image of two kittens in grass.User:<fake_token_around_image><image><fake_token_around_image>Describe this image.<end_of_utterance>Assistant:'


'User: Describe this image.Assistant: An image of two kittens in grass.User: Describe this image. Assistant: A dog wearing a pair of black glasses.'

# Show special characters injected around the images

In [12]:
generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
generated_text

'<s> User:<fake_token_around_image><image><fake_token_around_image> Describe this image.Assistant: An image of two kittens in grass.User:<fake_token_around_image><image><fake_token_around_image> Describe this image.<end_of_utterance> Assistant: A dog wearing a pair of black glasses.<end_of_utterance>'

# Ask Questions About Text in the Image

![](img/happy-car-chris.png)

In [5]:
from PIL import Image
img = Image.open("img/happy-car-chris.png") 

prompts = [
    "User: ",
    img,
    "Describe this image.",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt", debug=True).to(device)

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
generated_text

full_text='<s>User:<fake_token_around_image><image><fake_token_around_image>Describe this image.<end_of_utterance>Assistant:'


'<s> User:<fake_token_around_image><image><fake_token_around_image> Describe this image.<end_of_utterance> Assistant: A man is sitting on the hood of a white sports car.<end_of_utterance>'

In [8]:
from PIL import Image
img = Image.open("img/happy-car-chris.png") 

prompts = [
    "User: ",
    img,
    "Who makes this car?",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt", debug=True).to(device)

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
generated_text

full_text='<s>User:<fake_token_around_image><image><fake_token_around_image>Who makes this car?<end_of_utterance>Assistant:'


'<s> User:<fake_token_around_image><image><fake_token_around_image> Who makes this car?<end_of_utterance> Assistant: The car is made by Porsche.<end_of_utterance>'

![](img/baby-groot-toy.jpg)

In [9]:
img = Image.open("img/baby-groot-toy.jpg") 

prompts = [
    "User: ",
    img,
    "Which movie is this character from?",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt").to(device)

generated_ids = model.generate(**inputs, max_length=100) # eos_token_id=exit_condition, bad_words_ids=bad_words_ids)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

User: Which movie is this character from? Assistant: This character is from the movie Guardians of the Galaxy.


# Chain of thought

![](img/baby-groot-toy.jpg)

In [6]:
# This image is from https://www.amazon.com/Hot-Toys-Marvel-Guardians-Life-Size/dp/B07257N92P
img = Image.open("img/baby-groot-toy.jpg") 

prompts = [
    "User: ",
    img,
#    "Who produced the movie that features this character?",
    "Who produced the movie that features this character? Think step-by-step.",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt").to(device)

generated_ids = model.generate(**inputs, max_length=100) #, eos_token_id=exit_condition, bad_words_ids=bad_words_ids)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

User: Who produced the movie that features this character? Think step-by-step. Assistant: Marvel Studios produced the movie that features this character.


![](img/margherita-pizza.jpg)

In [7]:
# This image is from https://eu.ooni.com/blogs/recipes/margherita-pizza

img = Image.open("img/margherita-pizza.jpg") 

prompts = [
    "User: ",
    img,
    "How do I make this? Think step by step.",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt").to(device)

generated_ids = model.generate(**inputs, max_length=1000, eos_token_id=exit_condition, bad_words_ids=bad_words_ids)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

User: How do I make this? Think step by step. Assistant: To make a Margherita pizza at home, follow these steps:

1. Start by preheating your oven to the highest temperature it can reach, usually around 500°F (260°C).

2. On a lightly floured surface, roll out your pizza dough into a circle.

3. Transfer the dough to a pizza stone or baking sheet.

4. Sprinkle a layer of tomato sauce on top of the dough, leaving a small border around the edges.

5. Add a layer of fresh mozzarella cheese on top of the tomato sauce.

6. Add a layer of fresh basil leaves on top of the cheese.

7. Drizzle a small amount of olive oil over the pizza.

8. Place the pizza in the preheated oven and bake for 10-12 minutes, or until the crust is golden and the cheese is melted and bubbly.

9. Remove the pizza from the oven and let it cool for a few minutes before slicing and serving.

Optional: You can also add other toppings like sliced mushrooms, olives, or pepperoni to your Margherita pizza.
